In [ ]:
!pip3 install wandb

In [348]:
import pandas as pd
import wandb
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense
from wandb.keras import WandbCallback

In [349]:
train_data = pd.read_csv('/content/drive/MyDrive/csv_files/house_prices/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/csv_files/house_prices/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/csv_files/house_prices/sample_submission.csv')

In [350]:
wandb.init(project='House Prices - Kaggle')

best_epoch,199
best_val_loss,0.17072
epoch,199
loss,0.13367
val_loss,0.17072


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [351]:
train_data.head(20)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,...,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,...,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,...,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,...,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,

normalize data


In [352]:
# TRAIN DATA
train_data = train_data.replace(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], [0, 1, 2, 3, 4, 5]) 
train_data = train_data.replace(['Grvl', 'Pave'], [0, 1])
train_data = train_data.replace(['1Fam','2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], [0, 1, 2, 3, 4])
train_data = train_data.replace(['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'New', 'Oth', 'WD', 'ConLw'], [0, 1, 2, 3, 4, 5, 6, 7, 8])
train_data = train_data.replace(['Gtl','Mod', 'Sev'], [0, 1, 2])
train_data = train_data.replace(['Ex', 'Fa', 'Gd', 'Po', 'TA'], [0, 1, 2, 3, 4])
train_data = train_data.replace(['Reg','IR1','IR2','IR3'], [0, 1, 2, 3])
train_data = train_data.replace(['AllPub','NoSewr', 'NoSeWa', 'ELO'], [0, 1, 2, 3])
train_data = train_data.replace(['2Types','Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'], [0, 1, 2, 3, 4, 5])
train_data = train_data.replace(['BrkCmn', 'BrkFace', 'CBlock', 'Stone', 'None'], [0, 1, 2, 3, 4])
train_data = train_data.replace(['C (all)', 'FV', 'RH', 'RL', 'RM'], [0, 1, 2, 3, 4])
train_data = train_data.replace(['BrkTil','CBlock', 'PConc', 'Slab', 'Stone', 'Wood'], [0, 1, 2, 3, 4, 5])

# TEST DATA
test_data = test_data.replace(['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial'], [0, 1, 2, 3, 4, 5]) 
test_data = test_data.replace(['Grvl', 'Pave'], [0, 1])
test_data = test_data.replace(['1Fam','2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], [0, 1, 2, 3, 4])
test_data = test_data.replace(['COD', 'CWD', 'Con', 'ConLD', 'ConLI', 'New', 'Oth', 'WD', 'ConLw'], [0, 1, 2, 3, 4, 5, 6, 7, 8])
test_data = test_data.replace(['Gtl','Mod', 'Sev'], [0, 1, 2])
test_data = test_data.replace(['Ex', 'Fa', 'Gd', 'Po', 'TA'], [0, 1, 2, 3, 4])
test_data = test_data.replace(['Reg','IR1','IR2','IR3'], [0, 1, 2, 3])
test_data = test_data.replace(['AllPub','NoSewr', 'NoSeWa', 'ELO'], [0, 1, 2, 3])
test_data = test_data.replace(['2Types','Attchd', 'Basment', 'BuiltIn', 'CarPort', 'Detchd'], [0, 1, 2, 3, 4, 5])
test_data = test_data.replace(['BrkCmn', 'BrkFace', 'CBlock', 'Stone', 'None'], [0, 1, 2, 3, 4])
test_data = test_data.replace(['C (all)', 'FV', 'RH', 'RL', 'RM'], [0, 1, 2, 3, 4])
test_data = test_data.replace(['BrkTil','CBlock', 'PConc', 'Slab', 'Stone', 'Wood'], [0, 1, 2, 3, 4, 5])

In [353]:
train_data = train_data[['SaleCondition','Street', 'BldgType','SaleType','LandSlope','HeatingQC',
                   'Utilities','GarageType','MasVnrType','MSZoning','Foundation','YrSold',
                   'GarageCars','Fireplaces','TotRmsAbvGrd','BedroomAbvGr','KitchenAbvGr',
                   'FullBath','BsmtUnfSF', 'OverallCond', 'LotShape', 'MSSubClass', 'SalePrice']]

test_data = test_data[['SaleCondition','Street', 'BldgType','SaleType','LandSlope','HeatingQC',
                   'Utilities','GarageType','MasVnrType','MSZoning','Foundation','YrSold',
                   'GarageCars','Fireplaces','TotRmsAbvGrd','BedroomAbvGr','KitchenAbvGr',
                   'FullBath','BsmtUnfSF', 'OverallCond', 'LotShape', 'MSSubClass']]


train_data.head(20)

,SaleCondition,Street,BldgType,SaleType,LandSlope,HeatingQC,Utilities,GarageType,MasVnrType,MSZoning,Foundation,YrSold,GarageCars,Fireplaces,TotRmsAbvGrd,BedroomAbvGr,KitchenAbvGr,FullBath,BsmtUnfSF,OverallCond,LotShape,MSSubClass,SalePrice
0,4,1,0,7,0,0,0,1.0,1.0,3,2,2008,2,0,8,3,1,2,150,5,0,60,208500
1,4,1,0,7,0,0,0,1.0,4.0,3,2,2007,2,1,6,3,1,2,284,8,0,20,181500
2,4,1,0,7,0,0,0,1.0,1.0,3,2,2008,2,1,6,3,1,2,434,5,1,60,223500
3,0,1,0,7,0,2,0,5.0,4.0,3,0,2006,3,1,7,3,1,1,540,5,1,70,140000
4,4,1,0,7,0,0,0,1.0,1.0,3,2,2008,3,1,9,4,1,2,490,5,1,60,250000
5,4,1,0,7,0,0,0,1.0,4.0,3,5,2009,2,0,5,1,1,1,64,5,1,50,143000
6,4,1,0,7,0,0,0,1.0,3.0,3,2,2007,2,1,7,3,1,2,317,5,0,20,307000
7,4,1,0,7,0,0,0,1.0,3.0,3,2,2009,2,2,7,3,1,2,216,6,1,60,200000
8,0,1,0,7,0,2,0,5.0,4.0,4,0,2008,2,2,8,2,2,2,952,5,0,50,129900
9,4,1,1,7,0,0,0,1.0,4.0,3,0,2008,1,2,5,2,2,1,140,6,0,190,118000


In [354]:
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

train_data.head(20)

,SaleCondition,Street,BldgType,SaleType,LandSlope,HeatingQC,Utilities,GarageType,MasVnrType,MSZoning,Foundation,YrSold,GarageCars,Fireplaces,TotRmsAbvGrd,BedroomAbvGr,KitchenAbvGr,FullBath,BsmtUnfSF,OverallCond,LotShape,MSSubClass,SalePrice
0,4,1,0,7,0,0,0,1.0,1.0,3,2,2008,2,0,8,3,1,2,150,5,0,60,208500
1,4,1,0,7,0,0,0,1.0,4.0,3,2,2007,2,1,6,3,1,2,284,8,0,20,181500
2,4,1,0,7,0,0,0,1.0,1.0,3,2,2008,2,1,6,3,1,2,434,5,1,60,223500
3,0,1,0,7,0,2,0,5.0,4.0,3,0,2006,3,1,7,3,1,1,540,5,1,70,140000
4,4,1,0,7,0,0,0,1.0,1.0,3,2,2008,3,1,9,4,1,2,490,5,1,60,250000
5,4,1,0,7,0,0,0,1.0,4.0,3,5,2009,2,0,5,1,1,1,64,5,1,50,143000
6,4,1,0,7,0,0,0,1.0,3.0,3,2,2007,2,1,7,3,1,2,317,5,0,20,307000
7,4,1,0,7,0,0,0,1.0,3.0,3,2,2009,2,2,7,3,1,2,216,6,1,60,200000
8,0,1,0,7,0,2,0,5.0,4.0,4,0,2008,2,2,8,2,2,2,952,5,0,50,129900
9,4,1,1,7,0,0,0,1.0,4.0,3,0,2008,1,2,5,2,2,1,140,6,0,190,118000


In [357]:
X_train = train_data[['SaleCondition','Street','BldgType','SaleType','LandSlope','HeatingQC',
                   'Utilities','GarageType','MasVnrType','MSZoning','Foundation','YrSold',
                   'GarageCars','Fireplaces','TotRmsAbvGrd','BedroomAbvGr','KitchenAbvGr',
                   'FullBath','BsmtUnfSF', 'OverallCond', 'LotShape', 'MSSubClass']]

Y_train = train_data[['SalePrice']]

X_test = test_data
Y_test = submission[['SalePrice']]

In [358]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_val = np.array(X_val)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)
X_test = np.array(X_test)


In [359]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [360]:
model = tf.keras.models.Sequential([
    Dense(16, activation='relu'),
    Dense(32, activation='relu'),
    Dense(64, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='relu'),
])

In [361]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
              loss = tf.keras.losses.mean_squared_logarithmic_error)

In [362]:
model.fit(X_train, Y_train,
                validation_data=(X_val, Y_val),
                epochs=200,
                callbacks=[WandbCallback()]
                )

Epoch 1/200
37/37 [==============================] - 1s 6ms/step - loss: 110.5451 - val_loss: 39.7913
Epoch 2/200
37/37 [==============================] - 0s 2ms/step - loss: 34.0313 - val_loss: 28.6503
Epoch 3/200
37/37 [==============================] - 0s 2ms/step - loss: 25.2868 - val_loss: 21.3566
Epoch 4/200
37/37 [==============================] - 0s 3ms/step - loss: 18.7328 - val_loss: 15.7207
Epoch 5/200
37/37 [==============================] - 0s 2ms/step - loss: 13.8792 - val_loss: 11.6682
Epoch 6/200
37/37 [==============================] - 0s 2ms/step - loss: 10.1859 - val_loss: 8.4227
Epoch 7/200
37/37 [==============================] - 0s 2ms/step - loss: 7.3424 - val_loss: 6.0671
Epoch 8/200
37/37 [==============================] - 0s 2ms/step - loss: 5.2803 - val_loss: 4.2652
Epoch 9/200
37/37 [==============================] - 0s 2ms/step - loss: 3.5891 - val_loss: 2.7834
Epoch 10/200
37/37 [==============================] - 0s 2ms/step - loss: 2.3109 - val_loss: 1.76

In [363]:
print(X_test.shape,Y_test.shape)
model.evaluate(X_test, Y_test)

(1459, 22) (1459, 1)
46/46 [==============================] - 0s 1ms/step - loss: 0.0182


0.018167095258831978